In [1]:
import os
import numpy as np
import tensorflow as tf

In [2]:
# Set the random seed for reproducibility
random_seed = 42  # You can choose any seed value

# Set the random seed for NumPy
np.random.seed(random_seed)

# Set the random seed for TensorFlow
tf.random.set_seed(random_seed)

In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.metrics import Metric
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import pydicom
from sklearn.model_selection import train_test_split

In [4]:
tuberculosis_dir = r"C:\Users\kande\OneDrive\Desktop\TuberClosis\intbtr254"
normal_dir = r"C:\Users\kande\OneDrive\Desktop\TuberClosis\intbtr105\intbtr105"

In [5]:

def load_and_preprocess_images(directory, image_size=(224, 224)):
    images = []
    for filename in os.listdir(directory):
        if filename.endswith(".dicom"):
            dcm = pydicom.dcmread(os.path.join(directory, filename))
            image = dcm.pixel_array  # Get pixel data
            image = tf.image.resize(tf.expand_dims(image, axis=-1), image_size)  # Expand dimensions and resize
            image = tf.image.grayscale_to_rgb(image)  # Convert to RGB
            images.append(image)
    return np.array(images)


In [6]:
tuberculosis_images = load_and_preprocess_images(tuberculosis_dir)
normal_images = load_and_preprocess_images(normal_dir)


In [7]:
if len(tuberculosis_images) == 0 or len(normal_images) == 0:
    raise ValueError("Ensure that both classes have images in the specified directories.")

# Split data into training, validation, and test sets

In [8]:
X = np.concatenate((tuberculosis_images, normal_images))
y = np.concatenate((np.ones(len(tuberculosis_images)), np.zeros(len(normal_images))))

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=42)


In [10]:
class CustomAccuracyMetric(Metric):
    def __init__(self, name="custom_accuracy", **kwargs):
        super(CustomAccuracyMetric, self).__init__(name=name, **kwargs)
        self.total = self.add_weight("total", initializer="zeros")
        self.count = self.add_weight("count", initializer="zeros")

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_true = tf.cast(y_true, tf.float32)
        y_pred = tf.cast(y_pred, tf.float32)
        correct = tf.cast(tf.equal(y_true, tf.round(y_pred)), tf.float32)
        self.total.assign_add(tf.reduce_sum(correct))
        self.count.assign_add(tf.cast(tf.size(y_true), tf.float32))

    def result(self):
        return self.total / self.count


In [11]:


# Improved CNN Model Architecture
model = Sequential([
    Conv2D(32, (3, 3), activation="relu", input_shape=(224, 224, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation="relu"),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(64, activation="relu"),
    Dropout(0.5),  # Add Dropout layer
    BatchNormalization(),  # Add BatchNormalization layer
    Dense(1, activation="sigmoid")
])

# ... (Remaining code)



In [12]:
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=[CustomAccuracyMetric()])


In [13]:
datagen = ImageDataGenerator(rotation_range=40, width_shift_range=0.2, height_shift_range=0.2, shear_range=0.2, zoom_range=0.2, horizontal_flip=True, fill_mode="nearest")

early_stopping = EarlyStopping(monitor="val_custom_accuracy", patience=10, restore_best_weights=True)
model_checkpoint = ModelCheckpoint("best_model.h5", save_best_only=True)


In [14]:
history = model.fit(datagen.flow(X_train, y_train, batch_size=32), validation_data=(X_val, y_val), epochs=50, callbacks=[early_stopping, model_checkpoint])


Epoch 1/50
11/11 [==============================] - ETA: 0s - loss: 0.7353 - custom_accuracy: 0.5536

C:\Users\kande\anaconda3\Lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


11/11 [==============================] - 18s 1s/step - loss: 0.7353 - custom_accuracy: 0.5536 - val_loss: 0.8336 - val_custom_accuracy: 0.5814
Epoch 2/50
11/11 [==============================] - 19s 2s/step - loss: 0.6985 - custom_accuracy: 0.5681 - val_loss: 0.8135 - val_custom_accuracy: 0.6047
Epoch 3/50
11/11 [==============================] - 17s 2s/step - loss: 0.6799 - custom_accuracy: 0.5710 - val_loss: 0.8163 - val_custom_accuracy: 0.5814
Epoch 4/50
11/11 [==============================] - 24s 2s/step - loss: 0.6805 - custom_accuracy: 0.5652 - val_loss: 0.7671 - val_custom_accuracy: 0.5814
Epoch 5/50
11/11 [==============================] - 22s 2s/step - loss: 0.6479 - custom_accuracy: 0.6116 - val_loss: 0.6728 - val_custom_accuracy: 0.5814
Epoch 6/50
11/11 [==============================] - 17s 2s/step - loss: 0.6347 - custom_accuracy: 0.6319 - val_loss: 0.6317 - val_custom_accuracy: 0.5814
Epoch 7/50
11/11 [==============================] - 14s 1s/step - loss: 0.5855 - custom

In [15]:
test_loss, test_custom_accuracy = model.evaluate(X_test, y_test)
model.save("tuberculosis_detection_model.h5")


2/2 [==============================] - 1s 147ms/step - loss: 0.5993 - custom_accuracy: 0.7273


In [16]:
def detect_tuberculosis_single_image(model, image_path):
    # Read the DICOM image
    dcm = pydicom.dcmread(image_path)
    image = dcm.pixel_array

    # Ensure that the image has a channel dimension
    if len(image.shape) == 2:
        image = image[:, :, np.newaxis]

    # Expand dimensions to make it 4D
    image = tf.expand_dims(image, axis=0)

    # Resize the image to the model's input shape (224, 224, 3)
    image = tf.image.resize(image, (224, 224))
    
    # Convert to RGB
    image = tf.image.grayscale_to_rgb(image)
    image = image / 255.0  # Normalize pixel values if needed

    # Make a prediction using the model
    prediction = model.predict(image)

    # Interpret the prediction as needed for your specific model

    return prediction  # Adjust this line to return the appropriate result


In [17]:
result = detect_tuberculosis_single_image(model, r"C:\Users\kande\Downloads\intbtr254\intbtr254\cc730ec065ec6d9e85f2b5cf153d7fd9_Abnormal.dicom")

1/1 [==============================] - 0s 311ms/step


In [18]:
result = detect_tuberculosis_single_image(model, r"C:\Users\kande\Downloads\intbtr254\intbtr254\cc730ec065ec6d9e85f2b5cf153d7fd9_Abnormal.dicom")
# Assuming 'result' is a probability score
threshold = 0.5
if result >= threshold:
    print("Tuberculosis is detected")
else:
    print("Tuberculosis is not detected")
print(result)


1/1 [==============================] - 0s 74ms/step
Tuberculosis is detected
[[0.5304645]]
